In [1]:
import os
os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python"

In [2]:
from pyspark.sql.types import IntegerType, StringType, StructType, StructField 
import pyspark.sql.functions as F
from pyspark.context import SparkContext, SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col

In [3]:
conf = SparkConf()
conf = conf.setMaster("yarn")
conf = conf.set("spark.app.name", "recommend-ctr")
conf = conf.set("spark.executor.memory", "5g")
conf = conf.set("spark.driver.memory", "8g")
conf = conf.set("spark.driver.maxResultSize", "3g")
conf = conf.set("spark.executor.instances", "20")
conf = conf.set("spark.default.parallelism", "200")

In [4]:
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [5]:
OUTPUT_BUCKET_FOLDER = "hdfs:/user/lzhao/data/outbrain/preprocessed/"
DATA_BUCKET_FOLDER = "hdfs:/user/lzhao/data/outbrain/"
SPARK_TEMP_FOLDER = "hdfs:/user/lzhao/data/outbrain/spark-temp/"

In [6]:
events_schema = StructType(
                    [StructField("display_id", IntegerType(), True),
                    StructField("uuid_event", StringType(), True),                    
                    StructField("document_id_event", IntegerType(), True),
                    StructField("timestamp_event", IntegerType(), True),
                    StructField("platform_event", IntegerType(), True),
                    StructField("geo_location_event", StringType(), True)]
                    )

In [7]:
events_df = spark.read.schema(events_schema) \
  .options(header='true', inferschema='false', nullValue='\\N') \
  .csv(DATA_BUCKET_FOLDER + "events.csv") \
  .withColumn('day_event', (col('timestamp_event') / 1000 / 60 / 60 / 24).cast("int")) \
  .alias('events')  

In [8]:
events_df.show(5)

+----------+--------------+-----------------+---------------+--------------+------------------+---------+
|display_id|    uuid_event|document_id_event|timestamp_event|platform_event|geo_location_event|day_event|
+----------+--------------+-----------------+---------------+--------------+------------------+---------+
|         1|cb8c55702adb93|           379743|             61|             3|         US>SC>519|        0|
|         2|79a85fa78311b9|          1794259|             81|             2|         US>CA>807|        0|
|         3|822932ce3d8757|          1179111|            182|             2|         US>MI>505|        0|
|         4|85281d0a49f7ac|          1777797|            234|             2|         US>WV>564|        0|
|         5|8d0daef4bf5b56|           252458|            338|             2|             SG>00|        0|
+----------+--------------+-----------------+---------------+--------------+------------------+---------+
only showing top 5 rows



In [9]:
events_df.count()

23120126

In [10]:
print('Drop rows with empty "geo_location"...')
events_df = events_df.dropna(subset="geo_location_event")
events_df.count()

Drop rows with empty "geo_location"...


23119786

In [11]:
print('Drop rows with empty "platform"...')
events_df = events_df.dropna(subset="platform_event")
events_df.count()

Drop rows with empty "platform"...


23119781

In [12]:
promoted_content_schema = StructType(
  [StructField("ad_id", IntegerType(), True),
  StructField("document_id_promo", IntegerType(), True),                    
  StructField("campaign_id", IntegerType(), True),
  StructField("advertiser_id", IntegerType(), True)]
  )

In [13]:
promoted_content_df = spark.read.schema(promoted_content_schema) \
  .options(header='true', inferschema='false', nullValue='\\N') \
  .csv(DATA_BUCKET_FOLDER+"promoted_content.csv") \
  .alias('promoted_content')

In [14]:
promoted_content_df.show(5)

+-----+-----------------+-----------+-------------+
|ad_id|document_id_promo|campaign_id|advertiser_id|
+-----+-----------------+-----------+-------------+
|    1|             6614|          1|            7|
|    2|           471467|          2|            7|
|    3|             7692|          3|            7|
|    4|           471471|          2|            7|
|    5|           471472|          2|            7|
+-----+-----------------+-----------+-------------+
only showing top 5 rows



In [15]:
clicks_train_schema = StructType(
  [StructField("display_id", IntegerType(), True),
  StructField("ad_id", IntegerType(), True),                    
  StructField("clicked", IntegerType(), True)]
  )

In [16]:
clicks_train_df = spark.read.schema(clicks_train_schema) \
  .options(header='true', inferschema='false', nullValue='\\N') \
  .csv(DATA_BUCKET_FOLDER+"clicks_train.csv") \
  .alias('clicks_train')

In [22]:
clicks_train_df.count()

87141731

In [17]:
clicks_train_df.show(5)

+----------+------+-------+
|display_id| ad_id|clicked|
+----------+------+-------+
|         1| 42337|      0|
|         1|139684|      0|
|         1|144739|      1|
|         1|156824|      0|
|         1|279295|      0|
+----------+------+-------+
only showing top 5 rows



In [18]:
clicks_train_joined_df = clicks_train_df \
  .join(promoted_content_df, on='ad_id', how='left') \
  .join(events_df, on='display_id', how='left')
clicks_train_joined_df.createOrReplaceTempView('clicks_train_joined')

In [19]:
clicks_train_joined_df.take(2)

[Row(display_id=148, ad_id=152140, clicked=0, document_id_promo=1060089, campaign_id=19032, advertiser_id=1593, uuid_event='9adce6a5363308', document_id_event=1205772, timestamp_event=11202, platform_event=2, geo_location_event='US>LA>612', day_event=0),
 Row(display_id=148, ad_id=152656, clicked=0, document_id_promo=1086755, campaign_id=10511, advertiser_id=2151, uuid_event='9adce6a5363308', document_id_event=1205772, timestamp_event=11202, platform_event=2, geo_location_event='US>LA>612', day_event=0)]

In [20]:
validation_display_ids_df = clicks_train_joined_df.select('display_id','day_event') \
  .distinct() \
  .sampleBy("day_event", fractions={0: 0.2, 1: 0.2, 2: 0.2, 3: 0.2, 4: 0.2, \
  5: 0.2, 6: 0.2, 7: 0.2, 8: 0.2, 9: 0.2, 10: 0.2, 11: 1.0, 12: 1.0}, seed=0)
validation_display_ids_df.createOrReplaceTempView("validation_display_ids")
validation_set_df = spark.sql('''SELECT display_id, ad_id, uuid_event, day_event, 
  timestamp_event, document_id_promo, platform_event, geo_location_event 
  FROM clicks_train_joined t
    WHERE EXISTS (SELECT display_id FROM validation_display_ids 
      WHERE display_id = t.display_id)''')

In [21]:
validation_set_df.count()

27379904

In [21]:
validation_set_df.show(5)

+----------+------+--------------+---------+---------------+-----------------+--------------+------------------+
|display_id| ad_id|    uuid_event|day_event|timestamp_event|document_id_promo|platform_event|geo_location_event|
+----------+------+--------------+---------+---------------+-----------------+--------------+------------------+
|      2122| 36619|7ceed8e24a87d7|        0|         148795|           899906|             3|             SG>00|
|      2122| 81643|7ceed8e24a87d7|        0|         148795|          1094108|             3|             SG>00|
|      2122|216100|7ceed8e24a87d7|        0|         148795|          1548042|             3|             SG>00|
|      2659| 55819|964e40766c3f39|        0|         185389|           986576|             3|             CA>BC|
|      2659| 76816|964e40766c3f39|        0|         185389|           824972|             3|             CA>BC|
+----------+------+--------------+---------+---------------+-----------------+--------------+---

In [23]:
validation_set_gcs_output = "validation_set.parquet"
validation_set_df.coalesce(2).write.parquet(OUTPUT_BUCKET_FOLDER+validation_set_gcs_output, mode='overwrite')

In [24]:
spark.stop()